In [153]:
import dask
import xgboost as xgb
from Train_test_days_split import *

In [154]:
dask.config.set(temporary_directory='/global/home/users/qindan_zhu/myscratch/qindan_zhu/SatelliteNO2')

In [155]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
def get_slurm_dask_client(n_workers, n_cores):
    cluster = SLURMCluster(cores=n_cores,
                           memory='32GB',
                           project="co_aiolos",
                           walltime="02:00:00",
                           queue="savio2_gpu",
                           job_extra=['--gres=gpu:1','--cpus-per-task=2'])

    cluster.scale(n_workers)
    client = Client(cluster)
    return client
def get_slurm_dask_client_bigmem(n_nodes):
    cluster = SLURMCluster(cores=24,
                           memory='128GB',
                           project="co_aiolos",
                           walltime="02:00:00",
                           queue="savio2_bigmem",
                           job_extra=['--qos="savio_lowprio"'])

    cluster.scale(n_nodes*6)
    client = Client(cluster)
    return client
def get_slurm_dask_client_savio2(n_nodes):
    cluster = SLURMCluster(cores=12,
                           memory='64GB',
                           project="co_aiolos",
                           walltime="12:00:00",
                           queue="savio",
                           local_directory = '/global/home/users/qindan_zhu/myscratch/qindan_zhu/SatelliteNO2',
                           job_extra=['--qos="aiolos_savio_normal"'])

    cluster.scale(n_nodes*6)
    client = Client(cluster)
    return client
def get_slurm_dask_client_savio3(n_nodes):
    cluster = SLURMCluster(cores=32,
                           memory='96GB',
                           project="co_aiolos",
                           walltime="72:00:00",
                           queue="savio3",
                           job_extra=['--qos="aiolos_savio3_normal"'])

    cluster.scale(n_nodes*16)
    client = Client(cluster)
    return client

In [210]:
client = get_slurm_dask_client_savio2(4)

/global/home/users/qindan_zhu/PYTHON/SatelliteNO2/venv/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39421 instead
  http_address["port"], self.http_server.port


In [231]:
client

Client Scheduler: tcp://10.0.0.24:40260 Dashboard: http://10.0.0.24:39421/status,Cluster Workers: 2 Cores: 6 Memory: 32.00 GB


In [214]:
scheduler_info = client.scheduler_info()
workers = client.get_worker_logs()
list_of_workers = list(workers.keys())
list_of_workers

['tcp://10.0.1.128:33516',
 'tcp://10.0.1.128:34913',
 'tcp://10.0.1.128:42910',
 'tcp://10.0.1.128:46671',
 'tcp://10.0.1.130:34063',
 'tcp://10.0.1.130:35104',
 'tcp://10.0.1.130:38291',
 'tcp://10.0.1.130:42879',
 'tcp://10.0.1.133:33660',
 'tcp://10.0.1.133:36127',
 'tcp://10.0.1.133:44080',
 'tcp://10.0.1.133:45662',
 'tcp://10.0.1.71:32954',
 'tcp://10.0.1.71:34985',
 'tcp://10.0.1.71:35954',
 'tcp://10.0.1.71:42731',
 'tcp://10.0.1.79:35295',
 'tcp://10.0.1.79:38667',
 'tcp://10.0.1.79:40795',
 'tcp://10.0.1.79:45201',
 'tcp://10.0.1.95:39750',
 'tcp://10.0.1.95:41348',
 'tcp://10.0.1.95:41834',
 'tcp://10.0.1.95:43038']

In [ ]:
client.wait_for_workers(2)

In [163]:
client.has_what()

{'tcp://10.0.1.133:34333': (),
 'tcp://10.0.1.133:37651': (),
 'tcp://10.0.1.133:41908': (),
 'tcp://10.0.1.133:44986': (),
 'tcp://10.0.1.31:40986': (),
 'tcp://10.0.1.31:44805': (),
 'tcp://10.0.1.31:45540': (),
 'tcp://10.0.1.31:45921': (),
 'tcp://10.0.1.32:33752': (),
 'tcp://10.0.1.32:33945': (),
 'tcp://10.0.1.32:34455': (),
 'tcp://10.0.1.32:38314': (),
 'tcp://10.0.1.39:35911': (),
 'tcp://10.0.1.39:36289': (),
 'tcp://10.0.1.39:36671': (),
 'tcp://10.0.1.39:39178': (),
 'tcp://10.0.1.79:38544': (),
 'tcp://10.0.1.79:40809': (),
 'tcp://10.0.1.79:43618': (),
 'tcp://10.0.1.79:44767': (),
 'tcp://10.0.1.95:36652': (),
 'tcp://10.0.1.95:36910': (),
 'tcp://10.0.1.95:37412': (),
 'tcp://10.0.1.95:44838': ()}

In [16]:
#1.scheduler: tasks --> task graph 
#2.worker nodes

1. false dependence
b = a[i]+c
d = a[i+1]+c
compiler: b and d has to be processed in the same core with a saved
total_datasets = dict()
total_datasets['no2'] = no2 from ncdf
total_datasets['pres'] = pres from ncdf
each worker need memory to save total_datasets (full matrix)


2. nc.Datasets
data = read all data from ncdf file (~5G)
data['no2'] --> numpy array --> dask array (slow!!!)
package: xarray read in data from a ncdf file to a dask array (fast)

3. Solution
for each feature, call xarray, save the target feature data, and then do processing.##



SyntaxError: invalid syntax (<ipython-input-16-31320adfec79>, line 4)

In [215]:
import xarray as xr
import time
orig_filenames = sorted(glob(os.path.join(orig_file_path, 'met_conus*')))
orig_filenames = orig_filenames[0:16]
train_filenames, test_filenames = train_test_filename(orig_filenames)


In [216]:
train_filenames

['/global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_014',
 '/global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_015',
 '/global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_006',
 '/global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_008',
 '/global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_012',
 '/global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_009',
 '/global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_002',
 '/global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_013',
 '/global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_010',
 '/global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_005',
 '/global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_001',
 '/global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/m

In [217]:
nvel = 29
ngrid = 100250
ntime = 16
keep_indx = np.load('target_cells_index.npy')
reader = csv.reader(open("surrouding_cells.csv", "r"))
surr_arr = []
for row in reader:
    this_arr = list(map(int, row[1].strip(']|[').split(',')))
    surr_arr.append(this_arr)
surr_arr = np.array(surr_arr)[keep_indx, :]
def read_var_from_file_xr(filename, varname):
    ds = xr.open_mfdataset(filename, parallel=True)
    this_var = da.array(ds[varname][1:, keep_indx, :]).flatten()
    del ds
    return this_var
def read_var_from_file_nc(filename, varname):
    ds = nc.Dataset(filename)
    this_var = da.array(ds[varname][1:, keep_indx, :]).flatten()
    del ds
    return this_var
def read_2d_var_from_file_xr(filename, varname):
    ds = xr.open_mfdataset(filename, parallel=True)
    this_var = da.repeat(da.array(ds[varname][1:, keep_indx, :]),nvel,axis=2).flatten()
    del ds
    return this_var
def read_wind_from_file_xr(filename):
    ds = xr.open_mfdataset(filename, parallel=True)
    stg_u = da.array(ds['U'])
    stg_v = da.array(ds['V'])
    stg_w = da.array(ds['W'][1:, keep_indx, :])
    u_indx_left, u_indx_right, v_indx_bot, v_indx_up = find_indx_for_wind()
    wind_u = (stg_u[1:, u_indx_left, :] + stg_u[1:, u_indx_right, :])/2
    u = wind_u[:, keep_indx, :].flatten()
    wind_v = (stg_v[1:, v_indx_up, :] + stg_v[1:, v_indx_bot, :])/2
    v = wind_v[:, keep_indx, :].flatten()
    w = (stg_w[:, :, 1:] + stg_w[:, :, :-1])/2
    w = w.flatten()
    return da.stack([u,v,w])
def read_lightning_from_file_xr(filename):
    ds = xr.open_mfdataset(filename, parallel=True)
    cum_ic_flash = da.array(ds['IC_FLASHCOUNT'][:])
    cum_cg_flash = da.array(ds['CG_FLASHCOUNT'][:])
    ic_flash = da.repeat(cum_ic_flash[1:, :, :]-cum_ic_flash[:-1, :, :], nvel, axis=2)
    cg_flash = da.repeat(cum_cg_flash[1:, :, :]-cum_cg_flash[:-1, :, :], nvel, axis=2)
    e_lightning = ic_flash + cg_flash
    this_var = e_lightning[:, keep_indx, :].flatten()
    surrs = []
    surrs.append(this_var)
    for i in range(0, 24):
        surr_indx = surr_arr[:, i]
        surrs.append(e_lightning[:, surr_indx, :].flatten())
    return da.stack(surrs)
def read_anthro_emis_from_file_xr(filename):
    ds = xr.open_mfdataset(filename, parallel=True)
    e_no_lower = da.array(ds['E_NO'])[1:, :, :]
    e_no_upper = da.zeros((ntime, e_no_lower.shape[1], nvel - e_no_lower.shape[2]))
    e_no = da.concatenate([e_no_lower, e_no_upper], axis=2)
    e_no_total = []
    e_no_total.append(e_no[:, keep_indx, :].flatten())
    for i in range(0, 24):
        surr_indx = surr_arr[:, i]
        e_no_total.append(e_no[:, surr_indx, :].flatten())
    return da.stack(e_no_total)

In [218]:
def make_xgbmodel(client, filename):
    varnames = ['no2','pres','temp','CLDFRA']
    futures = []
    for varname in varnames:
        future = client.submit(read_var_from_file_xr, filename, varname)
        futures.append(future)
    dirt_feature = da.stack([future.result() for future in futures])
    client.cancel(futures)
    varnames_2d = ['COSZEN', 'PBLH', 'LAI', 'HGT', 'SWDOWN', 'GLW']
    futures_2d = []
    for varname in varnames_2d:
        future_2d = client.submit(read_2d_var_from_file_xr, filename, varname)
        futures_2d.append(future_2d)
    dirt2d_feature = da.stack([future.result() for future in futures_2d]) 
    client.cancel(futures_2d)
    wind_feature = client.submit(read_wind_from_file_xr, filename)
    anthroemis_future = client.submit(read_anthro_emis_from_file_xr, filename)
    lightning_future = client.submit(read_lightning_from_file_xr, filename)
    datasets = da.transpose(da.concatenate((dirt_feature, dirt2d_feature, wind_feature.result(), anthroemis_future.result(), lightning_future.result()),axis=0))
    client.cancel(dirt_feature) 
    client.cancel(dirt2d_feature)
    del wind_feature
    del anthroemis_future
    del lightning_future
    datasets_future = client.scatter(datasets)
    return datasets_future

In [219]:
client.has_what()

{'tcp://10.0.1.128:33516': (),
 'tcp://10.0.1.128:34913': (),
 'tcp://10.0.1.128:42910': (),
 'tcp://10.0.1.128:46671': (),
 'tcp://10.0.1.130:34063': (),
 'tcp://10.0.1.130:35104': (),
 'tcp://10.0.1.130:38291': (),
 'tcp://10.0.1.130:42879': (),
 'tcp://10.0.1.133:33660': (),
 'tcp://10.0.1.133:36127': (),
 'tcp://10.0.1.133:44080': (),
 'tcp://10.0.1.133:45662': (),
 'tcp://10.0.1.71:32954': (),
 'tcp://10.0.1.71:34985': (),
 'tcp://10.0.1.71:35954': (),
 'tcp://10.0.1.71:42731': (),
 'tcp://10.0.1.79:35295': (),
 'tcp://10.0.1.79:38667': (),
 'tcp://10.0.1.79:40795': (),
 'tcp://10.0.1.79:45201': (),
 'tcp://10.0.1.95:39750': (),
 'tcp://10.0.1.95:41348': (),
 'tcp://10.0.1.95:41834': (),
 'tcp://10.0.1.95:43038': ()}

In [220]:
create_total = True
for train_filename in train_filenames:
    print('Reading training data {}'.format(train_filename))
    this_dataset = make_xgbmodel(client, train_filename)
    if create_total:
        total_datasets = this_dataset.result()
        create_total = False
    else:
        total_datasets = da.concatenate((total_datasets, this_dataset.result()), axis=0)
    del this_dataset

Reading training data /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_014
Reading training data /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_015
Reading training data /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_006
Reading training data /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_008
Reading training data /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_012
Reading training data /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_009
Reading training data /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_002
Reading training data /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_013
Reading training data /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_010
Reading training data /global/home/users/qindan_zhu/myscratch/jlgrant/ML-

In [221]:
total_datasets

dask.array<concatenate, shape=(744256000, 63), dtype=float64, chunksize=(23258000, 1), chunktype=numpy.ndarray>

In [222]:
client.has_what()

{'tcp://10.0.1.128:33516': (),
 'tcp://10.0.1.128:34913': (),
 'tcp://10.0.1.128:42910': (),
 'tcp://10.0.1.128:46671': (),
 'tcp://10.0.1.130:34063': (),
 'tcp://10.0.1.130:35104': (),
 'tcp://10.0.1.130:38291': (),
 'tcp://10.0.1.130:42879': (),
 'tcp://10.0.1.133:33660': (),
 'tcp://10.0.1.133:36127': (),
 'tcp://10.0.1.133:44080': (),
 'tcp://10.0.1.133:45662': (),
 'tcp://10.0.1.71:32954': (),
 'tcp://10.0.1.71:34985': (),
 'tcp://10.0.1.71:35954': (),
 'tcp://10.0.1.71:42731': (),
 'tcp://10.0.1.79:35295': (),
 'tcp://10.0.1.79:38667': (),
 'tcp://10.0.1.79:40795': (),
 'tcp://10.0.1.79:45201': (),
 'tcp://10.0.1.95:39750': (),
 'tcp://10.0.1.95:41348': (),
 'tcp://10.0.1.95:41834': (),
 'tcp://10.0.1.95:43038': ()}

In [189]:
total_datasets = client.persist(total_datasets)

In [207]:
client.cancel(y)

In [209]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


In [223]:
X = total_datasets[:, 1:]
y = total_datasets[:, 0]


In [224]:
client.has_what()

{'tcp://10.0.1.128:33516': (),
 'tcp://10.0.1.128:34913': (),
 'tcp://10.0.1.128:42910': (),
 'tcp://10.0.1.128:46671': (),
 'tcp://10.0.1.130:34063': (),
 'tcp://10.0.1.130:35104': (),
 'tcp://10.0.1.130:38291': (),
 'tcp://10.0.1.130:42879': (),
 'tcp://10.0.1.133:33660': (),
 'tcp://10.0.1.133:36127': (),
 'tcp://10.0.1.133:44080': (),
 'tcp://10.0.1.133:45662': (),
 'tcp://10.0.1.71:32954': (),
 'tcp://10.0.1.71:34985': (),
 'tcp://10.0.1.71:35954': (),
 'tcp://10.0.1.71:42731': (),
 'tcp://10.0.1.79:35295': (),
 'tcp://10.0.1.79:38667': (),
 'tcp://10.0.1.79:40795': (),
 'tcp://10.0.1.79:45201': (),
 'tcp://10.0.1.95:39750': (),
 'tcp://10.0.1.95:41348': (),
 'tcp://10.0.1.95:41834': (),
 'tcp://10.0.1.95:43038': ()}

In [ ]:
del total_datasets

In [225]:
chunksize = int(X.shape[0]/len(client.nthreads())/3)
X = X.rechunk(chunks=(chunksize,62))
y = y.rechunk(chunks=(chunksize,1))

In [229]:
73/24

3.0416666666666665

In [226]:
X,y=client.persist([X,y])

In [227]:
client.has_what()

{'tcp://10.0.1.128:33516': ('open_dataset-659aa7e0368887c97ed8cd120a61279fE_NO-fffb6fefebb5e4d2f8cbfe3c94eab43c',
  'open_dataset-183052150a38fc5a3c66a9262a751044CG_FLASHCOUNT-3cba21dfd24e8b57fa370d6587c1a42c',
  'open_dataset-446163483d32b1946a079429b73b3585GLW-50439a822d6e02df358f09d690ee4be0',
  'open_dataset-29f00c2ded6196b25c588a24c4e402a3CLDFRA-154880d4089b33a3f15b6338ec120bfb',
  'open_dataset-d81cce4569d0e7ed69422fbee67454f1U-938d6c2c6c34c1948d6a2e426d95d073',
  'open_dataset-c328b05b7a7f598885c6aba2dc4871e1LAI-cf9ac3aa82faaa635415d5604ae5e659'),
 'tcp://10.0.1.128:34913': ('open_dataset-659aa7e0368887c97ed8cd120a61279fpres-959886f0231f5cdaef2a0a1658d627d1',
  'open_dataset-d81cce4569d0e7ed69422fbee67454f1temp-58cf52c9544230fc0c77415a075bcf81',
  'open_dataset-c328b05b7a7f598885c6aba2dc4871e1HGT-3ba6fddfb26c50b7e8f8ed691f2be1b8',
  'open_dataset-183052150a38fc5a3c66a9262a751044IC_FLASHCOUNT-ad0ab40580d2e526cbf1b22beffc51b0',
  'open_dataset-963e84e61d7145c121a9a81bb67de2daCG_FL

In [230]:
dtrain = xgb.dask.DaskDMatrix(client,X,y)

distributed.utils - ERROR - "('rechunk-merge-c310423d9bef51c9a13aacc9b50d3d35', 72, 0)"
Traceback (most recent call last):
  File "/global/home/users/qindan_zhu/PYTHON/SatelliteNO2/venv/lib/python3.6/site-packages/distributed/utils.py", line 655, in log_errors
    yield
  File "/global/home/users/qindan_zhu/PYTHON/SatelliteNO2/venv/lib/python3.6/site-packages/distributed/scheduler.py", line 2229, in add_worker
    typename=types[key],
KeyError: "('rechunk-merge-c310423d9bef51c9a13aacc9b50d3d35', 72, 0)"
distributed.core - ERROR - Exception while handling op register-worker
Traceback (most recent call last):
  File "/global/home/users/qindan_zhu/PYTHON/SatelliteNO2/venv/lib/python3.6/site-packages/distributed/core.py", line 517, in handle_comm
    result = await result
  File "/global/home/users/qindan_zhu/PYTHON/SatelliteNO2/venv/lib/python3.6/site-packages/distributed/scheduler.py", line 2229, in add_worker
    typename=types[key],
KeyError: "('rechunk-merge-c310423d9bef51c9a13aacc9b5

KeyboardInterrupt: 

In [195]:
total_datasets[:, 0]

dask.array<getitem, shape=(232580000,), dtype=float64, chunksize=(23258000,), chunktype=numpy.ndarray>

In [176]:
client

Client Scheduler: tcp://10.0.0.24:41781 Dashboard: http://10.0.0.24:42009/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [138]:
X,y = client.persist([X,y]) 

In [135]:
chunksize = int(X.shape[0]/len(client.nthreads())/3)
X = X.rechunk(chunks=(chunksize,62))
y = y.rechunk(chunks=(chunksize,1))

In [140]:
y

dask.array<rechunk-merge, shape=(232580000,), dtype=float64, chunksize=(3230277,), chunktype=numpy.ndarray>

In [143]:
client.who_has()

{'Array-2e3ddb17b7c9058404489f57fd7e020f': ('tcp://10.0.1.159:44908',),
 "('rechunk-merge-4a1ae8a1d8d15d367316844cbdd9cf3a', 40, 0)": (),
 "('rechunk-split-4a1ae8a1d8d15d367316844cbdd9cf3a', 363)": (),
 "('rechunk-merge-139f202830b6e9889a7fa1684db67622', 5, 3)": (),
 "('getitem-8175d06e4a860f3cf3bc5b6073fd4939', 5, 25)": (),
 "('concatenate-rechunk-merge-reshape-stack-transpose-getitem-8175d06e4a860f3cf3bc5b6073fd4939', 5, 25)": (),
 "('rechunk-split-44abcf6c1fc64f750358e0f4c2e666b7', 3)": ('tcp://10.0.1.144:41942',),
 "('getitem-5510d0e39a3a31132fc482b8b95a2e67', 0, 0, 1)": (),
 "('concatenate-bf65d9c299543b7811d74d673b5b66db', 0, 0, 1)": (),
 "('zeros-866fb3849026682faac0be26f070746f', 0, 0, 0)": (),
 "('rechunk-split-44abcf6c1fc64f750358e0f4c2e666b7', 2)": ('tcp://10.0.1.159:43673',),
 "('getitem-5510d0e39a3a31132fc482b8b95a2e67', 0, 0, 0)": (),
 "('concatenate-bf65d9c299543b7811d74d673b5b66db', 0, 0, 0)": ('tcp://10.0.1.159:43673',
  'tcp://10.0.1.159:44908',
  'tcp://10.0.1.159:45

In [144]:
client.has_what()


{'tcp://10.0.1.129:40536': ("('getitem-8175d06e4a860f3cf3bc5b6073fd4939', 7, 9)",
  "('getitem-8175d06e4a860f3cf3bc5b6073fd4939', 3, 8)",
  "('getitem-8175d06e4a860f3cf3bc5b6073fd4939', 1, 14)",
  "('rechunk-split-b6601f7cec5750c1bd9e004a58f8085d', 1)",
  "('getitem-8175d06e4a860f3cf3bc5b6073fd4939', 6, 9)",
  "('rechunk-split-2749a679a44c8516919ae4ded06ae222', 1)",
  "('getitem-8175d06e4a860f3cf3bc5b6073fd4939', 0, 14)",
  "('getitem-8175d06e4a860f3cf3bc5b6073fd4939', 2, 8)",
  "('getitem-8175d06e4a860f3cf3bc5b6073fd4939', 0, 0)",
  "('rechunk-split-2749a679a44c8516919ae4ded06ae222', 3)",
  "('rechunk-split-f3891e79bdba3aae10647512f2d165a9', 1)",
  "('getitem-8175d06e4a860f3cf3bc5b6073fd4939', 1, 0)",
  "('getitem-8175d06e4a860f3cf3bc5b6073fd4939', 5, 6)",
  "('getitem-8175d06e4a860f3cf3bc5b6073fd4939', 0, 1)",
  "('getitem-8175d06e4a860f3cf3bc5b6073fd4939', 4, 6)"),
 'tcp://10.0.1.129:41395': ("('getitem-8175d06e4a860f3cf3bc5b6073fd4939', 9, 38)",
  "('rechunk-split-4a1ae8a1d8d15d367

In [72]:
X

dask.array<getitem, shape=(232580000, 62), dtype=float64, chunksize=(23258000, 1), chunktype=numpy.ndarray>

In [76]:
client.has_what()

{'tcp://10.0.1.155:35690': ('Array-2e3ddb17b7c9058404489f57fd7e020f',
  'Array-2f44f7aed969582cc02eb2bb20f74780',
  'read_var_from_file_xr-305152514a6e2b2d7456ee94449dd160'),
 'tcp://10.0.1.155:41328': ('Array-2e3ddb17b7c9058404489f57fd7e020f',
  'Array-2f44f7aed969582cc02eb2bb20f74780',
  'read_var_from_file_xr-6ca5fef9328c145aff311fb1a805ad9f'),
 'tcp://10.0.1.155:44294': ('read_var_from_file_xr-6cf4207bc76400e127110e6238633457',
  'Array-2f44f7aed969582cc02eb2bb20f74780'),
 'tcp://10.0.1.155:46827': ('Array-2e3ddb17b7c9058404489f57fd7e020f',
  'read_var_from_file_xr-3be1f7031a6ef9539f9a72b39595c8f1'),
 'tcp://10.0.1.159:36257': ('Array-2f44f7aed969582cc02eb2bb20f74780',
  'Array-2e3ddb17b7c9058404489f57fd7e020f'),
 'tcp://10.0.1.159:38891': (),
 'tcp://10.0.1.159:41384': ('Array-2f44f7aed969582cc02eb2bb20f74780',
  'Array-2e3ddb17b7c9058404489f57fd7e020f'),
 'tcp://10.0.1.159:42149': ('Array-2f44f7aed969582cc02eb2bb20f74780',),
 'tcp://10.0.1.161:36827': ('Array-2e3ddb17b7c905840448

In [68]:
client.has_what()

{'tcp://10.0.1.155:35690': ('Array-2e3ddb17b7c9058404489f57fd7e020f',
  'Array-2f44f7aed969582cc02eb2bb20f74780',
  'read_var_from_file_xr-305152514a6e2b2d7456ee94449dd160'),
 'tcp://10.0.1.155:41328': ('Array-2e3ddb17b7c9058404489f57fd7e020f',
  'Array-2f44f7aed969582cc02eb2bb20f74780',
  'read_var_from_file_xr-6ca5fef9328c145aff311fb1a805ad9f'),
 'tcp://10.0.1.155:44294': ('read_var_from_file_xr-6cf4207bc76400e127110e6238633457',
  'Array-2f44f7aed969582cc02eb2bb20f74780'),
 'tcp://10.0.1.155:46827': ('Array-2e3ddb17b7c9058404489f57fd7e020f',
  'read_var_from_file_xr-3be1f7031a6ef9539f9a72b39595c8f1'),
 'tcp://10.0.1.159:36257': ('Array-2f44f7aed969582cc02eb2bb20f74780',
  'Array-2e3ddb17b7c9058404489f57fd7e020f'),
 'tcp://10.0.1.159:38891': (),
 'tcp://10.0.1.159:41384': ('Array-2f44f7aed969582cc02eb2bb20f74780',
  'Array-2e3ddb17b7c9058404489f57fd7e020f'),
 'tcp://10.0.1.159:42149': ('Array-2f44f7aed969582cc02eb2bb20f74780',),
 'tcp://10.0.1.161:36827': ('Array-2e3ddb17b7c905840448

In [79]:
client

Client Scheduler: tcp://10.0.0.24:42696 Dashboard: http://10.0.0.24:8787/status,Cluster Workers: 24 Cores: 72 Memory: 384.00 GB


In [78]:
client.who_has()

{'read_var_from_file_xr-305152514a6e2b2d7456ee94449dd160': ('tcp://10.0.1.155:35690',),
 'read_var_from_file_xr-6ca5fef9328c145aff311fb1a805ad9f': ('tcp://10.0.1.155:41328',),
 'read_var_from_file_xr-6cf4207bc76400e127110e6238633457': ('tcp://10.0.1.155:44294',),
 'read_var_from_file_xr-3be1f7031a6ef9539f9a72b39595c8f1': ('tcp://10.0.1.155:46827',),
 'read_2d_var_from_file_xr-52037c297090d55166d4b4d3ff340c5d': ('tcp://10.0.1.162:39032',),
 'read_2d_var_from_file_xr-389290e6155859d56cde7b9bed679528': ('tcp://10.0.1.162:44530',),
 'read_2d_var_from_file_xr-d739e0da48a5f5c0f3604ad23ce1a778': ('tcp://10.0.1.162:45290',),
 'read_2d_var_from_file_xr-31184f7fe30874b4758eb3e10a86189d': ('tcp://10.0.1.162:45840',),
 'read_2d_var_from_file_xr-7c1de1629bd4af1bbabfd432e04eff03': ('tcp://10.0.1.164:32905',),
 'read_2d_var_from_file_xr-c86b97049fe0a76669a830ad22ef8d73': ('tcp://10.0.1.164:36687',),
 'Array-2f44f7aed969582cc02eb2bb20f74780': ('tcp://10.0.1.155:41328',
  'tcp://10.0.1.159:41384',
  't

In [40]:
1. figure out what happended to rechunk 
2. how to allocate portions of X and y to each worker
3. how to ask dask to evenly distribute X and y across memory


NameError: name 'futures_2d' is not defined

In [17]:
#datasets = client.scatter(datasets)
datasets.result()

dask.array<transpose, shape=(46516000, 63), dtype=float64, chunksize=(23258000, 1), chunktype=numpy.ndarray>

In [16]:
datasets = da.transpose(da.concatenate((dirt_feature, dirt2d_feature, wind_feature.result(), anthroemis_future.result(), lightning_future.result()),axis=0))
X = datasets[:, 1:]
y = datasets[:, 0]
X,y = dask.persist(X,y)
del datasets

In [17]:
for future in futures:
    del future
for future in futures_2d:
    del future
del wind_feature
del anthroemis_future
del lightning_future

In [25]:
chunksize = int(X.shape[0]/60)
X = X.rechunk(chunks=(chunksize,62))
y = y.rechunk(chunks=(chunksize,1))

In [26]:
X

dask.array<rechunk-merge, shape=(46516000, 62), dtype=float64, chunksize=(775266, 62), chunktype=numpy.ndarray>

In [ ]:
client.shutdown()


In [ ]:
dtrain = xgb.dask.DaskDMatrix(client, X, y)

In [ ]:
output = xgb.dask.train(client,
                        {'verbosity': 2,
                         
                         'tree_method': 'hist',
                         'objective': 'reg:squarederror',
                         'subsample': 0.1,
                         'sampling_method': 'gradient_based'
                         },
                        dtrain,
                        num_boost_round=4, evals=[(dtrain, 'train')])

In [75]:
client

Client Scheduler: tcp://10.0.0.24:39571 Dashboard: http://10.0.0.24:8787/status,Cluster Workers: 40 Cores: 160 Memory: 480.00 GB


In [40]:
#create dtrain for xgboost
import dask_xgboost
params = {'objective': 'binary:logistic',
          'max_depth': 4, 'eta': 0.01, 'subsample': 0.5,
          'min_child_weight': 0.5}
bst = dask_xgboost.train(client, params, X, y, num_boost_round=10)

#datasets_future = client(client.scatter(datasets))
#dtrain_future = client.submit
#dtrain_future = client.submit(xgb.dask.DaskDMatrix, X, y)

ModuleNotFoundError: No module named 'dask_xgboost'

In [ ]:
import xgboost as xgb
def create_dtrain(datasets):
    X = datasets[:, 1,:]
    y = datasets[:, 0]
    dtrain = xgb.dask.DaskDMatrix(client, X, y)
    return dtrain

In [216]:
dtrain_future.result()

TypeError: Expecting <class 'distributed.client.Client'> or <class 'NoneType'>.  Got <class 'dask.array.core.Array'>

In [177]:
anthroemis_future.result()

dask.array<stack, shape=(25, 46516000), dtype=float64, chunksize=(1, 23258000), chunktype=numpy.ndarray>

In [168]:
varnames_2d = ['COSZEN', 'PBLH', 'LAI', 'HGT', 'SWDOWN', 'GLW']
futures_2d = []
for varname in varnames_2d:
    future_2d = client.submit(read_2d_var_from_file_xr, filename, varname)
    futures_2d.append(future_2d)
future_2d

<Future: pending, key: read_2d_var_from_file_xr-74b0c2381f36b9344e1ac5f6dd365148>

In [170]:
futures_2d[0].result()

dask.array<reshape, shape=(46516000,), dtype=float64, chunksize=(46516000,), chunktype=numpy.ndarray>

In [155]:

    
lightning_future = client.submit(read_lightning_from_file_xr, filename)

In [156]:
futures[-1].result()

dask.array<reshape, shape=(46516000,), dtype=float64, chunksize=(46516000,), chunktype=numpy.ndarray>

In [163]:
feature_v1 = da.transpose(da.stack([future.result() for future in futures]))
np.concatenate((feature_v1,lightning_future.result()),axis=1)

dask.array<concatenate, shape=(46516000, 30), dtype=float64, chunksize=(46516000, 1), chunktype=numpy.ndarray>

In [86]:
feature = client.submit(concatenate_features, futures)

In [88]:
feature.result()

AttributeError: 'Array' object has no attribute 'result'

In [77]:
features

dask.array<transpose, shape=(46516000, 4), dtype=float64, chunksize=(11629000, 1), chunktype=numpy.ndarray>

In [76]:
for future in futures:
    del future

In [29]:
#to do: surrounding lightning and vertical distribution of lightnings haven't been implemented yet
import time
orig_filenames = sorted(glob(os.path.join(orig_file_path, 'met_conus*')))
orig_filenames = orig_filenames[0:5]
train_filenames, test_filenames = train_test_filename(orig_filenames)
x_total = []
y_total = []
add_total= []
starttime = time.time()
for train_filename in train_filenames:
    print('Reading in data from {}'.format(train_filename))
    additional_arr, x_arr, y_arr, x_labels, additional_features = read_orig_file_from_wrf(train_filename)
    add_total.append(additional_arr)
    x_total.append(x_arr)
    y_total.append(y_arr)
print("--- {} minutes ---".format((time.time() - starttime)/60))

Reading in data from /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_002
Reading in data from /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_003
Reading in data from /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_005
Reading in data from /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_004
Reading in data from /global/home/users/qindan_zhu/myscratch/jlgrant/ML-WRF/ML-WRF/met_conus_2005_001
--- 0.25605957508087157 minutes ---


In [ ]:
future = client.compute(x_total)
client.close()

In [ ]:
future[0].result()

In [ ]:
#prepare matrix for ml
import xgboost as xgb
starttime = time.time()
dtrain = xgb.dask.DaskDMatrix(client, da.array(x_total), da.array(y_total))
print("--- {} minutes ---".format((time.time() - starttime)/60))

In [ ]:
client()

In [5]:
#create a file saving the index of target cells
create_index_list_for_target_cells()

In [3]:
#create a file saving the indexes of surrounding emissions
create_index_dict_for_surrouding_emis()

Make surrouding cells for cell lat 0
Make surrouding cells for cell lat 1
Make surrouding cells for cell lat 2
Make surrouding cells for cell lat 3
Make surrouding cells for cell lat 4
Make surrouding cells for cell lat 5
Make surrouding cells for cell lat 6
Make surrouding cells for cell lat 7
Make surrouding cells for cell lat 8
Make surrouding cells for cell lat 9
Make surrouding cells for cell lat 10
Make surrouding cells for cell lat 11
Make surrouding cells for cell lat 12
Make surrouding cells for cell lat 13
Make surrouding cells for cell lat 14
Make surrouding cells for cell lat 15
Make surrouding cells for cell lat 16
Make surrouding cells for cell lat 17
Make surrouding cells for cell lat 18
Make surrouding cells for cell lat 19
Make surrouding cells for cell lat 20
Make surrouding cells for cell lat 21
Make surrouding cells for cell lat 22
Make surrouding cells for cell lat 23
Make surrouding cells for cell lat 24
Make surrouding cells for cell lat 25
Make surrouding cells 

Make surrouding cells for cell lat 216
Make surrouding cells for cell lat 217
Make surrouding cells for cell lat 218
Make surrouding cells for cell lat 219
Make surrouding cells for cell lat 220
Make surrouding cells for cell lat 221
Make surrouding cells for cell lat 222
Make surrouding cells for cell lat 223
Make surrouding cells for cell lat 224
Make surrouding cells for cell lat 225
Make surrouding cells for cell lat 226
Make surrouding cells for cell lat 227
Make surrouding cells for cell lat 228
Make surrouding cells for cell lat 229
Make surrouding cells for cell lat 230
Make surrouding cells for cell lat 231
Make surrouding cells for cell lat 232
Make surrouding cells for cell lat 233
Make surrouding cells for cell lat 234
Make surrouding cells for cell lat 235
Make surrouding cells for cell lat 236
Make surrouding cells for cell lat 237
Make surrouding cells for cell lat 238
Make surrouding cells for cell lat 239
Make surrouding cells for cell lat 240
Make surrouding cells for

In [4]:
print('additional features: {}'.format(additional_features))
print('features: {}'.format(x_labels))

additional features: ['xlon', 'xlat', 'date', 'elev', 'hour', 'IC_FLASHCOUNT', 'CG_FLASHCOUNT']
features: ['COSZEN', 'U', 'V', 'W', 'PBLH', 'E_NO', 'pres', 'temp', 'LAI', 'HGT', 'CLDFRA', 'SWDOWN', 'GLW', 'anthro_surr_emis_00', 'lightning_surr_emis_00', 'anthro_surr_emis_01', 'lightning_surr_emis_01', 'anthro_surr_emis_02', 'lightning_surr_emis_02', 'anthro_surr_emis_03', 'lightning_surr_emis_03', 'anthro_surr_emis_04', 'lightning_surr_emis_04', 'anthro_surr_emis_05', 'lightning_surr_emis_05', 'anthro_surr_emis_06', 'lightning_surr_emis_06', 'anthro_surr_emis_07', 'lightning_surr_emis_07', 'anthro_surr_emis_08', 'lightning_surr_emis_08', 'anthro_surr_emis_09', 'lightning_surr_emis_09', 'anthro_surr_emis_10', 'lightning_surr_emis_10', 'anthro_surr_emis_11', 'lightning_surr_emis_11', 'anthro_surr_emis_12', 'lightning_surr_emis_12', 'anthro_surr_emis_13', 'lightning_surr_emis_13', 'anthro_surr_emis_14', 'lightning_surr_emis_14', 'anthro_surr_emis_15', 'lightning_surr_emis_15', 'anthro_sur

In [5]:
x_total[0][0]

dask.array<reshape, shape=(46516000,), dtype=float32, chunksize=(46516000,), chunktype=numpy.MaskedArray>

In [6]:
client = Client()

In [7]:
client

Client Scheduler: tcp://127.0.0.1:61441 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [12]:
#save_arr = da.array(save_arr.T)
client = get_slurm_dask_client(1, 1)

/global/home/users/qindan_zhu/PYTHON/SatelliteNO2/venv/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44606 instead
  http_address["port"], self.http_server.port


In [15]:
client

Client Scheduler: tcp://10.0.0.24:42736 Dashboard: http://10.0.0.24:44606/status,Cluster Workers: 1 Cores: 1 Memory: 32.00 GB


In [ ]:
#fit ml model
starttime = time.time()
output = xgb.dask.train(client,
                        {'verbosity': 2,
                         'tree_method': 'hist',
                         'objective': 'reg:squarederror'
                         },
                        dtrain,
                        num_boost_round=4, evals=[(dtrain, 'train')])
print("--- {} minutes ---".format((time.time() - starttime)/60))

In [34]:
cluster = SLURMCluster(cores=24, memory = '128 GB', project='co_aiolos',queue='savio2_bigmem',walltime='02:00:00')

/global/home/users/qindan_zhu/PYTHON/SatelliteNO2/venv/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36835 instead
  http_address["port"], self.http_server.port


In [35]:
cluster.scale(1)

In [36]:
client = Client(cluster)

In [38]:
client

Client Scheduler: tcp://10.0.0.24:43513 Dashboard: http://10.0.0.24:36835/status,Cluster Workers: 0 Cores: 0 Memory: 0 B
